<a href="https://colab.research.google.com/github/AhmadIsmail83N/MyData/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip show tensorflow

# Reinstall Tnsorflow if you have v2.*

In [0]:
!pip uninstall -y tensorflow

In [0]:
!pip install tensorflow-gpu==1.15

**Restart runtime** by button and launch 

# Lab executing

In [0]:
import tensorflow as tf
from tensorflow . python . keras . datasets import cifar10
from tensorflow . examples . tutorials . mnist import input_data
import numpy as np
import time

In [0]:
cifar10 . load_data ()

In [0]:

def weight_variable ( shape ):
  initial = tf. truncated_normal (shape , stddev =0.1)
  return tf. Variable ( initial )

def bias_variable ( shape ):
  initial = tf. constant (0.1 , shape = shape )
  return tf. Variable ( initial )

def conv2d (x, W):
  return tf.nn. conv2d (x, W, strides =[1 , 1, 1, 1], padding ='SAME')

def max_pool_2x2 (x):
  return tf.nn. max_pool (x, ksize =[1 , 2, 2, 1],
  strides =[1 , 2, 2, 1], padding ='SAME')
  
def conv_layer (input , shape ):
  W = weight_variable ( shape )
  b = bias_variable ([ shape [3]])
  return tf.nn. relu ( conv2d (input , W) + b)

def full_layer (input , size ):
  in_size = int ( input . get_shape ()[1])
  W = weight_variable ([ in_size , size ])
  b = bias_variable ([ size ])
  return tf. matmul (input , W) + b

In [0]:
x = tf. placeholder (tf. float32 , shape =[ None , 784])
y_= tf. placeholder (tf. float32 , shape =[ None , 10])
keep_prob = tf. placeholder (tf. float32 )
x_image = tf. reshape (x, [-1, 28, 28, 1])

In [0]:
with tf. name_scope ('conv_1'):
  conv1 = conv_layer ( x_image , shape =[5 , 5, 1, 32])
  conv1_pool = max_pool_2x2 (conv1)
  
with tf. name_scope ('conv_2'):
  conv2 = conv_layer ( conv1_pool , shape =[5 , 5, 32, 64])
  conv2_pool = max_pool_2x2 ( conv2 )
  #
with tf. name_scope ('conv_3'):
  conv3 = conv_layer ( conv2_pool , shape =[5 , 5, 64, 64])
  conv3_pool = max_pool_2x2 ( conv3 )
  conv3_flat = tf. reshape ( conv3_pool , [-1, 4*4*64] )

with tf. name_scope ('full_1'):
  full_1 = tf.nn. relu ( full_layer ( conv3_flat , 1024))

with tf. name_scope ('dropout'):
  full1_drop = tf.nn. dropout ( full_1 , keep_prob = keep_prob )

with tf. name_scope ('activations'):
  y_conv = full_layer ( full1_drop , 10)
  tf. summary . scalar ('cross_entropy_loss',y_conv )
  mnist = input_data. read_data_sets ('tmp \ data', one_hot = True )

with tf. name_scope ('cross'):
  cross_entropy = tf. reduce_mean (tf.nn. softmax_cross_entropy_with_logits_v2
                                    (logits= y_conv, labels =y_))
# # SGD
train_step = tf. train . AdagradOptimizer ( learning_rate =1e-4). minimize ( cross_entropy )
correct_prediction = tf. equal (tf. argmax ( y_conv , 1), tf. argmax (y_ , 1))
accuracy = tf. reduce_mean (tf. cast ( correct_prediction , tf. float32 ))

In [0]:
with tf. Session () as sess :
  sess . run (tf. global_variables_initializer ())
  start_time = time . time ()
  for i in range (50000):
    batch = mnist . train . next_batch (50)
    if i % 100 == 0:
      train_accuracy = sess . run ( accuracy ,feed_dict ={x: batch [0] , 
                                      y_: batch [1] , keep_prob : 1.0})
      print (" time {}, step {}, training accuracy {}". format ( time .time() - start_time,i,train_accuracy ) )
    sess . run ( train_step , feed_dict ={x: batch [0] , y_: batch [1] , keep_prob : 0.5})

  X= mnist . test . images . reshape (10 , 1000 , 784)
  Y= mnist . test . labels . reshape (10 , 1000 , 10)
  test_accuracy = np. mean ([ sess . run ( accuracy , feed_dict ={x:X[i], y_:Y[i],
  keep_prob :1.0}) for i in range (10)])
  print (" test accuracy : {}". format ( test_accuracy ))
